In [50]:
#import numpy
import numpy as np

In [51]:
#Initial set up

# Define the one-hot encoding function
def one_hot_encode(char, chars):
    vocab_size = len(chars)
    encoded = np.zeros(vocab_size)
    encoded[chars.index(char)] = 1
    return encoded.reshape(-1, 1)  #to change array in a vector

# Define the vocabulary
chars = ['l', 'e', 'a', 'r', 'n']

# Test the function
for i in chars:
    print(one_hot_encode(i,chars).shape)

    
# Define the RNN parameters
input_size = len(chars)
hidden_size = 5  # Adjust as needed
output_size = len(chars)
learning_rate = 1

parameters = np.load(r"C:\Users\Dell\Documents\IITM\IITM7\DL\parameters.npz")
U= parameters.get("U")
bu= parameters.get("bu")
W= parameters.get("W")
bw= parameters.get("bw")
V= parameters.get("V")
bv= parameters.get("bv")
np.savez("parameters", U=U, bu=bu, W=W, bw=bw, V=V, bv=bv)

print(U.shape, W.shape, V.shape)

(5, 1)
(5, 1)
(5, 1)
(5, 1)
(5, 1)
(5, 5) (5, 5) (5, 5)


In [52]:
def tanh(x):
    return np.tanh(x)

# Define derivative of tanh function
def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Subtracting the maximum value for numerical stability
    return exp_x / np.sum(exp_x, axis=0)

def softmax_derivative(x):
    s = softmax(x)
    return np.diagflat(s) - np.outer(s, s)

In [53]:
'''#FirstPass and first step (to understand)

sprev = np.zeros((hidden_size, 1))  # Initial hidden state

# Compute the input at the current time step
x = one_hot_encode('l', chars)  # Reshape to a column vector
input_contribution = np.dot(U, x)

# Compute the contribution from the previous hidden state
hidden_contribution = np.dot(W, sprev)

# Add input and hidden contributions, and bias term to obtain the current hidden state
s = input_contribution + hidden_contribution 

# Apply tanh activation function to the current hidden state
s = np.tanh(s)

# Print the shapes for understanding
print("Shape of input contribution:", input_contribution.shape)
print("Shape of hidden contribution:", hidden_contribution.shape)
print("Shape of current hidden state:", s.shape)
print(s)
print(s.shape)
print(np.sum(s))

O=np.dot(V,s)
print(O, O.shape, np.sum(O))

# Compute the softmax probabilities
softmax_probs = softmax(O)

# Print the softmax probabilities
print("Softmax probabilities:", softmax_probs)

# Define the true probability distribution (one-hot encoded for 'e')
true_probs = one_hot_encode('e', chars)

# Compute the cross-entropy loss
loss = -np.sum(true_probs * np.log(softmax_probs))

# Print the loss
print("Cross-entropy loss:", loss)'''

'#FirstPass and first step (to understand)\n\nsprev = np.zeros((hidden_size, 1))  # Initial hidden state\n\n# Compute the input at the current time step\nx = one_hot_encode(\'l\', chars)  # Reshape to a column vector\ninput_contribution = np.dot(U, x)\n\n# Compute the contribution from the previous hidden state\nhidden_contribution = np.dot(W, sprev)\n\n# Add input and hidden contributions, and bias term to obtain the current hidden state\ns = input_contribution + hidden_contribution \n\n# Apply tanh activation function to the current hidden state\ns = np.tanh(s)\n\n# Print the shapes for understanding\nprint("Shape of input contribution:", input_contribution.shape)\nprint("Shape of hidden contribution:", hidden_contribution.shape)\nprint("Shape of current hidden state:", s.shape)\nprint(s)\nprint(s.shape)\nprint(np.sum(s))\n\nO=np.dot(V,s)\nprint(O, O.shape, np.sum(O))\n\n# Compute the softmax probabilities\nsoftmax_probs = softmax(O)\n\n# Print the softmax probabilities\nprint("Softm

In [54]:
input_sequence = chars
total_loss = 0
true_label_s5 = np.zeros((output_size, 1))

# Initialize dictionaries to store values at each time step
x = {}
s = {}
loss = {}

sprev = np.array([0,0,0,0,0]).reshape(-1,1)
s[0]=sprev
true_y = {}
y_hat = {}

# Perform forward propagation and compute loss for each time step
for i in range(len(input_sequence)):  # Iterate through each character
    # Compute input at current time step
    x[i] = one_hot_encode(input_sequence[i], chars)
    
    # Compute hidden state
    current_s = np.tanh(np.dot(U, x[i]) + np.dot(W, sprev))
    s[i+1] = current_s
    
    # Compute output probabilities and true label
    if i < len(input_sequence) - 1:
        # Compute output probabilities for all but the last character
        O = np.dot(V, current_s)
        current_y_hat = softmax(O)
        y_hat[i] = current_y_hat
        
        # True label for the next character in the sequence
        true_y[i] = one_hot_encode(input_sequence[i + 1], chars)
        
        # Compute loss at this time step
        loss1 = -np.sum(true_y[i] * np.log(current_y_hat))
        loss[i]=loss1
        # Accumulate total loss
        total_loss += loss1
        print("Loss at step", i + 1, "is:", total_loss)
    
    else:
        # For the last character in the sequence
        # Compute output probabilities for the last character
        O = np.dot(V, current_s)
        current_y_hat = softmax(O)
        y_hat[i] = current_y_hat
        
        # True label for the last character is given as true_label_s5
        true_y[i] = true_label_s5
        
        # Compute loss at the last step
        loss1 = -np.sum(true_label_s5 * np.log(current_y_hat))
        loss[i]=loss1
        total_loss += loss1
        print("Loss at last step is", total_loss)

    # Update previous hidden state for next time step
    sprev = current_s


# Print total loss
print("Total loss:", total_loss)

# Now you have dictionaries x, s, true_y, and y_hat containing values for each time step
for i in s:
    print(i, s[i])
for i in y_hat:
    print(i, y_hat[i])

Loss at step 1 is: 2.400343465148997
Loss at step 2 is: 6.620953965241555
Loss at step 3 is: 8.733205261817997
Loss at step 4 is: 12.28157916678833
Loss at last step is 12.28157916678833
Total loss: 12.28157916678833
0 [[0]
 [0]
 [0]
 [0]
 [0]]
1 [[ 0.45952909]
 [-0.22995027]
 [-0.43286563]
 [-0.50967271]
 [ 0.89874415]]
2 [[ 0.41653184]
 [ 0.99049789]
 [-0.85940849]
 [-0.99800625]
 [ 0.72569196]]
3 [[ 0.99917404]
 [ 0.99856167]
 [ 0.04620508]
 [-0.99477806]
 [ 0.49342697]]
4 [[ 0.99957558]
 [-0.15172561]
 [-0.89362858]
 [-0.9969986 ]
 [-0.92411435]]
5 [[-0.66968424]
 [ 0.97496617]
 [ 0.63936819]
 [-0.93785853]
 [-0.80746262]]
0 [[0.81847983]
 [0.0906868 ]
 [0.00570932]
 [0.04127048]
 [0.04385356]]
1 [[0.8941071 ]
 [0.02571633]
 [0.01468967]
 [0.03647585]
 [0.02901105]]
2 [[0.34680734]
 [0.27718645]
 [0.05566301]
 [0.12096533]
 [0.19937786]]
3 [[0.10665259]
 [0.08107662]
 [0.11114889]
 [0.67235051]
 [0.02877139]]
4 [[0.03782231]
 [0.03585545]
 [0.27509607]
 [0.09915536]
 [0.55207081]]


In [55]:
for i in loss:
    print(i, loss[i])
    
for i in s:
    print(i, s[i])
    
for i in true_y:
    print(i, true_y[i])
    
for i in y_hat:
    print(i, y_hat[i])
    
for i in x:
    print(i, x[i])
    
print(len(s))

0 2.400343465148997
1 4.220610500092558
2 2.112251296576441
3 3.548373904970334
4 -0.0
0 [[0]
 [0]
 [0]
 [0]
 [0]]
1 [[ 0.45952909]
 [-0.22995027]
 [-0.43286563]
 [-0.50967271]
 [ 0.89874415]]
2 [[ 0.41653184]
 [ 0.99049789]
 [-0.85940849]
 [-0.99800625]
 [ 0.72569196]]
3 [[ 0.99917404]
 [ 0.99856167]
 [ 0.04620508]
 [-0.99477806]
 [ 0.49342697]]
4 [[ 0.99957558]
 [-0.15172561]
 [-0.89362858]
 [-0.9969986 ]
 [-0.92411435]]
5 [[-0.66968424]
 [ 0.97496617]
 [ 0.63936819]
 [-0.93785853]
 [-0.80746262]]
0 [[0.]
 [1.]
 [0.]
 [0.]
 [0.]]
1 [[0.]
 [0.]
 [1.]
 [0.]
 [0.]]
2 [[0.]
 [0.]
 [0.]
 [1.]
 [0.]]
3 [[0.]
 [0.]
 [0.]
 [0.]
 [1.]]
4 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
0 [[0.81847983]
 [0.0906868 ]
 [0.00570932]
 [0.04127048]
 [0.04385356]]
1 [[0.8941071 ]
 [0.02571633]
 [0.01468967]
 [0.03647585]
 [0.02901105]]
2 [[0.34680734]
 [0.27718645]
 [0.05566301]
 [0.12096533]
 [0.19937786]]
3 [[0.10665259]
 [0.08107662]
 [0.11114889]
 [0.67235051]
 [0.02877139]]
4 [[0.03782231]
 [0.03585545]
 [0.2750

In [56]:
# Back Propagation
# Initialize gradients
grad_U = np.zeros_like(U)
grad_V = np.zeros_like(V)
grad_W = np.zeros_like(W)
grad_b = np.zeros_like(bu)
grad_c = np.zeros_like(bv)

loss=0
# Initialize d_h_next
d_s_next = np.zeros((hidden_size, 1))

for t in reversed(range(1, len(s))):  # Iterate from the last time step backwards
    #print("step", t)   
    '''In this network there are 5 hidden layers
    At s(5) feed from s(4) and x(4), and pred_y=y_hat[4], and true_y=y[4]'''
    # At s(5), there is no loss as true_y[4] =[0 0 0 0 0]
    # Compute cross-entropy loss (as a scalar)
    #print(y_hat[t-1], true_y[t-1])
    loss += -np.sum(np.log(y_hat[t-1]) * true_y[t-1])
    #print(loss)
    # Backpropagate into output (derivative of cross-entropy)
    d_o = y_hat[t-1] - true_y[t-1]
    
    # Backpropagate into V
    grad_V += np.dot(d_o, s[t].T)
    #print(grad_V)
    # Backpropagate into s
    d_s = np.dot(V.T, d_o) + d_s_next  # Sum of implicit and explicit loss)
    #print(d_s)                                   #for last layer d_s_next is all 0s
    # Backpropagate through non-linearity
    d_h =  d_s * (1 - s[t]**2)
    #print(d_h)
    grad_U += np.dot(d_h, x[t-1].T)

    grad_W += np.dot(d_h, s[t-1].T)
    #print("Loss gradient wrt W at present layer is", grad_W)
    # Update d_s_next for the next iteration
    d_s_next = np.dot(W.T, d_h)
    #print("Loss Flwoing back from this layer is", d_s_next)
print(grad_W, grad_W.sum())
    # Update weights and biases using gradients
U -= learning_rate * grad_U
V -= learning_rate * grad_V
W -= learning_rate * grad_W

#print(learning_rate)
#print(W, W.sum())

[[ 1.46439149e-02  9.06365905e-03 -9.75266134e-03 -9.62839780e-03
  -1.15541539e-01]
 [ 2.15911541e-01  2.08395239e-01 -8.00368924e-04 -2.13087787e-01
   3.63776824e-02]
 [ 5.92922012e-02  2.19958265e-02 -4.52899269e-01 -1.96689787e-01
  -7.92012029e-01]
 [ 8.52400553e-03 -7.64296584e-03 -2.84259913e-03 -3.44889969e-03
  -2.57198334e-02]
 [ 6.88004285e-01  6.27930474e-01 -1.56018113e+00 -1.54669826e+00
   3.00403427e+00]] -0.04277242516305568


In [64]:
#Prediction now,  ``Auto-regressive mode'' (that is, 
#the input to the next time step is the output from the previous time step)
# Initialize the initial hidden state
sprev = np.zeros((hidden_size, 1))

# Initialize a list to store the predicted characters
predicted_sequence = []

# Iterate through each time step in the sequence
x_current = one_hot_encode(input_sequence[0], chars)
for i in range(len(input_sequence)):
    # Compute input at current time step
    
    
    # Compute hidden state for the current time step using the updated weights
    current_s = np.tanh(np.dot(U, x_current) + np.dot(W, sprev))
    
    # Compute output probabilities for the current time step using the updated weights
    O = np.dot(V, current_s)
    current_y_hat = softmax(O)
    if i == 4:
        print(x_current , current_y_hat) 
    # Get the index of the predicted character (argmax of output probabilities)
    predicted_index = np.argmax(current_y_hat)
    
    # Get the predicted character from the vocabulary
    predicted_char = chars[predicted_index]
    
    # Append the predicted character to the list of predicted sequence
    predicted_sequence.append(predicted_char)
    
    # Update the hidden state for the next time step
    
    x_current = np.int8(current_y_hat >=current_y_hat.max() )
    sprev = current_s
    if i == 4:
        print(x_current , current_s)
# Join the predicted characters to form the predicted sequence
predicted_sequence_str = ''.join(predicted_sequence)

# Print the predicted sequence
print("Predicted sequence:", predicted_sequence_str)

[[0]
 [0]
 [0]
 [1]
 [0]] [[0.01765745]
 [0.08256517]
 [0.03294536]
 [0.06506616]
 [0.80176586]]
[[0]
 [0]
 [0]
 [0]
 [1]] [[ 0.99965191]
 [-0.93944717]
 [-0.81241202]
 [-0.99048735]
 [-0.99988941]]
Predicted sequence: enlrn
